In [2]:
# SQL Alchemy

# psycopg2

In [37]:
import pandas as pd
import psycopg2 as pg



In [45]:
# Credentials
host = 'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com'
port = 5432
database = 'comunidadedsdb'
username = 'member'
pwd = 'cdspa'

In [46]:
conn = pg.connect( user = username,
                   password=pwd,
                   host=host,
                   port=port,
                   database=database )

# 1.0. Schema Query

In [21]:
cursor = conn.cursor()

query_schema = """
    SELECT nspname
    FROM pg_catalog.pg_namespace
"""

cursor.execute( query_schema )
record = cursor.fetchall()
cursor.close()
conn.close()

# 2.0. Table Query

In [31]:
cursor = conn.cursor()

query_tables = """
    SELECT tablename
    FROM pg_tables
    WHERE schemaname='pa004'
"""

cursor.execute( query_tables )

record = cursor.fetchall()
print( record )

cursor.close()
conn.close()

[('users',), ('vehicle',), ('insurance',)]


# 3.0. Collect Data using Psycopg2

In [35]:
cursor = conn.cursor()

query_tables_users = """
    SELECT *
    FROM pa004.users u
    WHERE u.age > 44
    LIMIT 10
"""

cursor.execute( query_tables_users )

record = cursor.fetchall()
print( record )

cursor.close()
conn.close()

[(2, 'Male', 76, 3.0, 26.0), (3, 'Male', 47, 28.0, 26.0), (8, 'Female', 56, 28.0, 26.0), (11, 'Female', 47, 35.0, 124.0), (14, 'Male', 76, 28.0, 13.0), (15, 'Male', 71, 28.0, 30.0), (20, 'Female', 60, 33.0, 124.0), (21, 'Male', 65, 28.0, 124.0), (22, 'Male', 49, 28.0, 124.0), (27, 'Female', 51, 28.0, 124.0)]


In [40]:
data = pd.DataFrame( record )
data.head()

,0,1,2,3,4
0,2,Male,76,3.0,26.0
1,3,Male,47,28.0,26.0
2,8,Female,56,28.0,26.0
3,11,Female,47,35.0,124.0
4,14,Male,76,28.0,13.0


# 4.0. Collect Data using Pandas

In [41]:
import pandas.io.sql as psql

In [49]:
query_tables_users = """
    SELECT *
    FROM pa004.users u LEFT JOIN pa004.vehicle v ON ( u.id = v.id )
                       LEFT JOIN pa004.insurance i ON ( u.id = i.id )

    WHERE u.age > 44
"""

df = psql.read_sql( query_tables_users, conn )
df.head()

/Users/thiago/Thiago/Data_Science/comunidade_ds/pa04_health_insurance/health_04_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,gender,age,region_code,policy_sales_channel,id,driving_license,vehicle_age,vehicle_damage,id,previously_insured,annual_premium,vintage,response
0,8,Female,56,28.0,26.0,8,1,1-2 Year,Yes,8,0,32031.0,72,1
1,11,Female,47,35.0,124.0,11,1,1-2 Year,Yes,11,0,47576.0,46,1
2,39,Male,45,8.0,124.0,39,1,1-2 Year,Yes,39,0,42297.0,264,0
3,42,Female,54,28.0,122.0,42,1,1-2 Year,No,42,0,38560.0,184,0
4,53,Female,49,3.0,152.0,53,1,1-2 Year,No,53,1,25553.0,130,0


In [51]:
df1 = pd.read_sql( query_tables_users, conn )
df1 = df1.drop( df.columns[[ 5,9]], axis=1 )
df1.head()

/Users/thiago/Thiago/Data_Science/comunidade_ds/pa04_health_insurance/health_04_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,Female,56,28.0,26.0,1,1-2 Year,Yes,0,32031.0,72,1
1,Female,47,35.0,124.0,1,1-2 Year,Yes,0,47576.0,46,1
2,Male,45,8.0,124.0,1,1-2 Year,Yes,0,42297.0,264,0
3,Female,54,28.0,122.0,1,1-2 Year,No,0,38560.0,184,0
4,Female,49,3.0,152.0,1,1-2 Year,No,1,25553.0,130,0
